In [ ]:
import yaml as yml
import pandas as pd
import math
import re

In [ ]:
with open("config/config.yaml", "r") as f:
    config = yml.safe_load(f)

In [ ]:
config["dna_paired_samples_tsv"]

In [ ]:
dna_meta = pd.read_csv(config["dna_paired_samples_tsv"], sep="\t", header=None, comment="#")

In [ ]:
rna_meta = pd.read_csv(config["rna_paired_samples_tsv"], sep="\t", header=None, comment="#")

In [ ]:
# Make a list of metrics files that Dragen generates as part of the Tamor pipeline for somatic dna and rna analysis

# DNA germline, includes some files generated for the normal during paired tumor-normal analysis. Excludes some duplicate
# file generated by same (e.g. .dna.somatic.wgs_contig_mean_cov_normal.csv is same as .dna.germline.wgs_overall_mean_cov.csv).
dna_germline_csv_metric_file_suffices = [
                                     ".dna.germline.ploidy_estimation_metrics.csv",
                                     ".dna.germline.roh_metrics.csv",
                                     ".dna.germline.time_metrics.csv", 
                                     ".dna.germline.vc_hethom_ratio_metrics.csv", 
                                     ".dna.germline.vc_metrics.csv", 
                                     ".dna.germline.wgs_coverage_metrics.csv", 
                                     ".dna.germline.wgs_hist.csv",
                                     ".dna.germline.wgs_overall_mean_cov.csv",
                                     ".dna.germline.sv_metrics.csv",
                                     ".dna.germline.trimmer_metrics.csv",
#                                     ".dna.germline.fastqc_metrics.csv",
                                     ".dna.germline.cnv_metrics.csv",
                                     ".dna.somatic.hla_metrics.csv"]

# These germline-related files will have a different prefix since they are generated during the somatic analysis.
dna_germline_normal_csv_metric_file_suffices = [
                                     ".dna.somatic.tmb_overall_mean_cov_normal.csv",
                                     ".dna.somatic.tmb_coverage_metrics_normal.csv"]

# Single out some metrics files for special processing because they have very large dimensions, but only some values need to be retained, 
# e.g. per-contig metrics that include all the decoys, alt contigs, etc. but we will just report the reference autosomal, sex and mito contigs
dna_germline_contig_csv_metric_file_suffices = [".dna.germline.mapping_metrics.csv",
                                                ".dna.germline.wgs_contig_mean_cov.csv"]

dna_germline_contig_normal_csv_metric_file_suffices = [".dna.somatic.tmb_contig_mean_cov_normal.csv"]
                                            
# Note that we are not capturing histogram bin metrics files as these cause major column bloat, e.g. ".dna.germline.fragment_length_hist.csv"
dna_somatic_csv_metric_file_suffices = [".dna.somatic.cnv_metrics.csv", 
                                         ".dna.somatic.ploidy_estimation_metrics.csv",
                                         ".dna.somatic.sv_metrics.csv",
                                         ".dna.somatic.time_metrics.csv",
                                         ".dna.somatic.vc_metrics.csv",
                                         ".dna.somatic.wgs_overall_mean_cov_tumor.csv",
                                         ".dna.somatic.wgs_coverage_metrics_tumor.csv",
                                         ".dna.somatic.tmb_overall_mean_cov_tumor.csv",
                                         ".dna.somatic.tmb_coverage_metrics_tumor.csv",
                                         ".dna.somatic.hla_metrics.tumor.csv",
                                         ".dna.somatic.tmb.metrics.csv",
#                                         ".dna.somatic.fastqc_metrics.csv",
                                         ".dna.somatic.trimmer_metrics.csv"
                                       ]

dna_somatic_contig_csv_metric_file_suffices = [".dna.somatic.mapping_metrics.csv", 
                                                ".dna.somatic.wgs_contig_mean_cov_tumor.csv",
                                                ".dna.somatic.tmb_contig_mean_cov_tumor.csv"]

# The following files require special procesing as they have their own format: 
# .dna.germline.insert-stats.tab, 
# .dna.somatic.insert-stats.tab, 
# .dna.somatic.hrdscore.csv,
# .dna.somatic.microsat_output.json, 
# and .dna.somatic.cnv.vcf.gz (for ModelSource=DEGENERATE_DIPLOID, etc.)

rna_csv_metric_file_suffices = [".rna.fastqc_metrics.csv",
                                ".rna.fusion_metrics.csv",
                                ".rna.mapping_metrics.csv",
                                ".rna.wgs_overall_mean_cov.csv",
                                ".rna.wgs_coverage_metrics.csv",
                                ".rna.wgs_coverage_metrics.csv",
                                ".rna.trimmer_metrics.csv",
                                ".rna.time_metrics.csv"]

In [ ]:
metrics = []
hhmmss_regex = re.compile('(\\d\\d):(\\d\\d):(\\d\\d)')

In [ ]:
# Every subject's tumor-normal pair is allowed to have one value per column.
# We only generate stats for the samples that are in the DNA or RNA sample files.
for metadata_index,metadata_row in dna_meta.iterrows():
    subject = metadata_row[0] 
    print(subject)
    # Pull out the metadata row that corresponds to the subject
    tumor = metadata_row[1]
    project = metadata_row[9]

    tumor_normal_pair_metrics = {}
    tumor_normal_pair_metrics["subject"] = subject
    tumor_normal_pair_metrics["project"] = project
    tumor_normal_pair_metrics["tumor"] = tumor
    # See if there is a germline sample, and load the metrics if they exist.
    if metadata_row[3] != "NO_NORMAL": 
        germline = metadata_row[3]
        tumor_normal_pair_metrics["normal"] = germline
        for suffix in dna_germline_csv_metric_file_suffices:
            filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+germline+suffix
            #print(f"Checking for {filepath}")
            if os.path.isfile(filepath):
                print("germline dna: "+filepath)
                # Some metrics files have some rows with 5 columns, where the 5th is a % measure vs an absolute number in the 4th column.
                # Passing in 5 names resolves a default problem of parsing a ragged CSV file. 
                data_rows = pd.read_csv(filepath, sep=",", names=range(5), header=None)
                for data_index,data_row in data_rows.iterrows():
                    # print (",".join([data_row[0],str(data_row[1])]))
                    # Make an exception for the histogram CSV files where we only have 2 columns
                    if isinstance(data_row[2], float) and math.isnan(data_row[2]):
                        metric_column_name = "Normal " + data_row[0]
                        tumor_normal_pair_metrics[metric_column_name] = data_row[1]
                    # FASTQC metrics get labelled as Read1 or Read2 related
                    elif isinstance(data_row[1], str) and data_row[1].startswith("Read"):
                        metric_column_name = "Normal "+data_row[0]+" "+data_row[1]+" "+data_row[2]
                        tumor_normal_pair_metrics[metric_column_name] = data_row[3]
                    else:
                        metric_column_name = "Normal "+data_row[0]+" "+data_row[2]
                        value = data_row[3]
                        # Transform all time intervals into seconds for plotting.
                        if isinstance(value, str):
                            m = hhmmss_regex.match(value)
                            if m:
                                value = int(m.group(1))*3600+int(m.group(2))*60+int(m.group(3))
                        tumor_normal_pair_metrics[metric_column_name] = value
                    
        for suffix in dna_germline_normal_csv_metric_file_suffices:
            filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+tumor+"_"+germline+suffix
            if os.path.isfile(filepath):
                print("germline dna: "+filepath)
                data_rows = pd.read_csv(filepath, sep=",", names=range(5), header=None)
        for suffix in dna_germline_contig_csv_metric_file_suffices:
            filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+germline+suffix
            if os.path.isfile(filepath):
                print("germline dna contigs: "+filepath)                
        for suffix in dna_germline_contig_normal_csv_metric_file_suffices:
            filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+tumor+"_"+germline+suffix
            if os.path.isfile(filepath):
                print("germline dna contigs: "+filepath)
    
    # Assume there is a somatic sample, load metrics if they exist. Tumor-only is not yet supported.
    for suffix in dna_somatic_csv_metric_file_suffices:
        filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+tumor+"_"+germline+suffix
        #print(f"Checking for {filepath}")
        if os.path.isfile(filepath):
            print("somatic dna: "+filepath)
    for suffix in dna_somatic_contig_csv_metric_file_suffices:
        filepath = config["output_dir"]+"/"+project+"/"+subject+"/"+subject+"_"+tumor+"_"+germline+suffix
        #print(f"Checking for {filepath}")
        if os.path.isfile(filepath):
            print("somatic dna contigs: "+filepath)
    metrics.append(tumor_normal_pair_metrics)

# Only the first RNA sample found in the config file for any given subject normal will be included in the stats.
rna_matched = {}
for index,row in rna_meta.iterrows():
    subject = row[0] 
    print(subject)
    project = row[3]
    rna = row[1]
    matching_tumor_dna = row[2]
    if matching_tumor_dna in rna_matched.keys(): # Ignore subsequent RNA samples associated with a tumor.
        continue
    else:
        rna_matched[matching_tumor_dna] = True
        
    for suffix in rna_csv_metric_file_suffices:
        filepath = config["output_dir"]+"/"+project+"/"+subject+"/rna/"+subject+"_"+rna+suffix
        #print(f"Checking for {filepath}")
        if os.path.isfile(filepath):
            print("rna: "+filepath)

# Specially-formatted files that need their own processing.
# .dna.germline.insert-stats.tab, 
# .dna.somatic.insert-stats.tab, 
            
# .dna.somatic.hrdscore.csv,
# .dna.somatic.microsat_output.json, 
# and .dna.somatic.cnv.vcf.gz (for ModelSource=DEGENERATE_DIPLOID, etc.)

In [ ]:
metrics_dataframe = pd.DataFrame.from_dict(metrics)
metrics_dataframe.to_csv("resources/metrics/dragen.tsv", sep="\t")